In [1]:
import pathlib
import tensorflow  as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import layers,models
import matplotlib.pyplot as plt

In [2]:
!ls /home/mw/input/99119154/people_data/people_data/48-data

Angelina Jolie	   Johnny Depp	      Nicole Kidman	  Tom Hanks
Brad Pitt	   Kate Winslet       Robert Downey Jr	  Will Smith
Denzel Washington  Leonardo DiCaprio  Sandra Bullock
Hugh Jackman	   Megan Fox	      Scarlett Johansson
Jennifer Lawrence  Natalie Portman    Tom Cruise


## 学习如何使用pathlib

In [3]:
path = pathlib.Path('/home/mw/input/99119154/people_data/people_data/48-data')

In [4]:
path.parent # 父级

PosixPath('/home/mw/input/99119154/people_data/people_data')

In [5]:
path.name # 名称

'48-data'

In [6]:
images = list(path.rglob('*.jpg')) # 使用rglob递归获取全部图片,这里使用正则表达式来获取

In [7]:
print(len(images))

1800


In [8]:
from PIL import Image

In [9]:
image = Image.open(images[9])
print(image.width,image.height) # 获取图片的宽和高

474 568


In [10]:
batch_size = 32 # 设置batch_szie
img_height = 224
img_width = 224

## 设置训练集与验证集

In [11]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path,
    validation_split=0.1,
    subset="training",
    label_mode = "categorical", # 标签将被编码为分类向量，一共17个类别
    seed=123, # 随机种子的设置
    image_size=(img_height, img_width), # 将图片重新resize
    batch_size=batch_size) # batch_size的大小


Found 1800 files belonging to 17 classes.
Using 1620 files for training.


In [12]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path,
    validation_split=0.1,
    subset="validation",
    label_mode = "categorical",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)


Found 1800 files belonging to 17 classes.
Using 180 files for validation.


In [13]:
class_names = train_ds.class_names
print(class_names) # 查看分类

['Angelina Jolie', 'Brad Pitt', 'Denzel Washington', 'Hugh Jackman', 'Jennifer Lawrence', 'Johnny Depp', 'Kate Winslet', 'Leonardo DiCaprio', 'Megan Fox', 'Natalie Portman', 'Nicole Kidman', 'Robert Downey Jr', 'Sandra Bullock', 'Scarlett Johansson', 'Tom Cruise', 'Tom Hanks', 'Will Smith']


In [14]:
for image,labels in train_ds: # 查看数据集个数
    print(image.shape)
    print(labels.shape)
    break

(32, 224, 224, 3)
(32, 17)


In [15]:
AUTOTUNE = tf.data.AUTOTUNE

In [16]:
#设置预加载
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [31]:
model = models.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)), # 卷积层1，卷积核3*3  
    layers.AveragePooling2D((2, 2)),               # 池化层1，2*2采样
    layers.Conv2D(32, (3, 3), activation='relu'),  # 卷积层2，卷积核3*3
    layers.AveragePooling2D((2, 2)),               # 池化层2，2*2采样
    layers.Dropout(0.5),  
    layers.Conv2D(64, (3, 3), activation='relu'),  # 卷积层3，卷积核3*3
    layers.AveragePooling2D((2, 2)),     
    layers.Dropout(0.5),  
    layers.Conv2D(128, (3, 3), activation='relu'),  # 卷积层3，卷积核3*3
    layers.Dropout(0.5), 
    
    layers.Flatten(),                       # Flatten层，连接卷积层与全连接层
    layers.Dense(128, activation='relu'),   # 全连接层，特征进一步提取
    layers.Dense(len(class_names))               # 输出层，输出预期结果
])

model.summary()  # 打印网络结构

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 222, 222, 16)      448       
_________________________________________________________________
average_pooling2d (AveragePo (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      4640      
_________________________________________________________________
average_pooling2d_1 (Average (None, 54, 54, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        1

In [32]:
optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [33]:

epochs = 100

# 保存最佳模型参数
checkpointer = ModelCheckpoint('best_model.h5',
                                monitor='val_accuracy',
                                verbose=1,
                                save_best_only=True,
                                save_weights_only=True)

In [34]:
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=50
                   ,callbacks=[checkpointer]) # 这里要将callbacks传入参数，否则保存没有效果。

Epoch 1/50
51/51 [==============================] - 8s 47ms/step - loss: 2.8659 - accuracy: 0.1117 - val_loss: 2.7818 - val_accuracy: 0.1389

Epoch 00001: val_accuracy improved from -inf to 0.13889, saving model to best_model.h5
Epoch 2/50
51/51 [==============================] - 1s 26ms/step - loss: 2.7472 - accuracy: 0.1190 - val_loss: 2.6347 - val_accuracy: 0.0833

Epoch 00002: val_accuracy did not improve from 0.13889
Epoch 3/50
51/51 [==============================] - 1s 27ms/step - loss: 2.4912 - accuracy: 0.1929 - val_loss: 2.4912 - val_accuracy: 0.2389

Epoch 00003: val_accuracy improved from 0.13889 to 0.23889, saving model to best_model.h5
Epoch 4/50
51/51 [==============================] - 1s 27ms/step - loss: 2.2640 - accuracy: 0.2605 - val_loss: 2.3687 - val_accuracy: 0.2500

Epoch 00004: val_accuracy improved from 0.23889 to 0.25000, saving model to best_model.h5
Epoch 5/50
51/51 [==============================] - 1s 26ms/step - loss: 2.0098 - accuracy: 0.3570 - val_loss:

In [35]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(loss))

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

<Figure size 864x288 with 2 Axes>

In [38]:
model.load_weights('./best_model.h5')

## 使用VGG
vgg使用3*3卷积核，以及更深的网络。vgg的特点是每层输出的时候把w,h减半，channel增加翻倍(图片来自CSDN)
![vgg图片](https://img-blog.csdnimg.cn/20190725104625128.png) 

In [47]:
!pwd

/home/mw/project


实现一个vgg-A，也就是9层的神经网络

In [32]:
vgg_9 = models.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255,input_shape=(img_height, img_width, 3)),
    layers.Conv2D(64, (3, 3),padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Conv2D(128,(3,3),padding='same',activation='relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Conv2D(256,(3,3),padding='same',activation='relu'),
    layers.Conv2D(256,(3,3),padding='same',activation='relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Conv2D(512,(3,3),padding='same',activation='relu'),
    layers.Conv2D(512,(3,3),padding='same',activation='relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Conv2D(512,(3,3),padding='same',activation='relu'),
    layers.Conv2D(512,(3,3),padding='same',activation='relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Dropout(0.7),
    layers.Flatten(), 
    layers.Dense(4096,activation='relu'),
    #layers.Dropout(0.8),# 训练的时候发现太容易过拟合了，全连接层加了一个dropout防止过拟合
    layers.Dense(4096,activation='relu'),
    layers.Dense(len(class_names)),
    
])

In [33]:
# optimizer = tf.keras.optimizers.Adam(lr=0.01)
# vgg_9.compile(optimizer=optimizer,
#               loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])
# 经过不断调试发现优化器使用Adam loss不下降accuracy不提升。改用sgd优化器。
vgg_9.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])


In [34]:
checkpointer = ModelCheckpoint('vgg_best_model.h5',
                                monitor='val_accuracy',
                                verbose=1,
                                save_best_only=True,
                                save_weights_only=True)

In [35]:
vgg_history = vgg_9.fit(train_ds,
                    validation_data=val_ds,
                    epochs=50,callbacks=[checkpointer])

Epoch 1/50
51/51 [==============================] - 12s 231ms/step - loss: 2.8298 - accuracy: 0.1034 - val_loss: 2.8049 - val_accuracy: 0.1389

Epoch 00001: val_accuracy improved from -inf to 0.13889, saving model to vgg_best_model.h5
Epoch 2/50
51/51 [==============================] - 11s 225ms/step - loss: 2.8256 - accuracy: 0.1001 - val_loss: 2.7960 - val_accuracy: 0.1389

Epoch 00002: val_accuracy did not improve from 0.13889
Epoch 3/50
51/51 [==============================] - 12s 227ms/step - loss: 2.8022 - accuracy: 0.1179 - val_loss: 2.7876 - val_accuracy: 0.1389

Epoch 00003: val_accuracy did not improve from 0.13889
Epoch 4/50
51/51 [==============================] - 12s 228ms/step - loss: 2.7895 - accuracy: 0.1163 - val_loss: 2.7117 - val_accuracy: 0.1667

Epoch 00004: val_accuracy improved from 0.13889 to 0.16667, saving model to vgg_best_model.h5
Epoch 5/50
51/51 [==============================] - 11s 225ms/step - loss: 2.7448 - accuracy: 0.1202 - val_loss: 2.8219 - val_acc

In [36]:
acc = vgg_history.history['accuracy']
val_acc = vgg_history.history['val_accuracy']

loss = vgg_history.history['loss']
val_loss = vgg_history.history['val_loss']

epochs_range = range(len(loss))

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

<Figure size 864x288 with 2 Axes>

# 总结：
1.当loss和accuracy不下降时，进行问题排查：
模型结构是否有问题
权重的初始化
你的数据和label是否正确
优化器的使用是否合理
损失函数是否正确
学习率是否过大
这里进行尝试，最开始用的adm优化器，换成了sgd优化器。

2.正则化尝试：
	这里主要试验了dropout正则化，发现当Dropout设置过大时，会造成欠拟合问题。